In [47]:
import pandas as pd
import requests
import numpy as np

In [48]:
data = pd.read_csv("C:/Users/arneg/Desktop/Google ML Immersion/URL PROJECT/SSH Public/HiddenFraudulentURLs.csv",delimiter=";")

______________________________________________________________________________________________

<b>1) DUPLICATES SERVE NO PURPOSE.  EVEN THOUGH CONTENT LENGTH CHANGES, NO DIFFERENCES</b>

In [49]:
df_rd = data.drop_duplicates()

In [50]:
df_rd['url2'] = df_rd['url']

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
test = df_rd.groupby(['url','isHiddenFraudulent']).url2.count().reset_index().sort_values('url2',ascending=False)

In [ ]:
test[(test['url2'] > 1) & (test['isHiddenFraudulent'] == True)]

In [ ]:
df_rd[df_rd['url'] == "http://www.kinnaree.co.uk/x.txt"]

______________________________________________________________________________________________


<b>2) CONTENT LENGTH SEEMS TO HAVE SIGNIFICANCE</b>

In [ ]:
df_rd.groupby(['compromissionType','isHiddenFraudulent']).contentLength.mean().reset_index()

In [ ]:
df_rd.groupby(['compromissionType','isHiddenFraudulent']).contentLength.describe()

______________________________________________________________________________________________


<b>3) Powered By That Causes the Most Shenanigans</b>

In [ ]:
powered = df_rd.groupby(['poweredBy','isHiddenFraudulent']).url.count().reset_index().sort_values(['isHiddenFraudulent','url'])

In [ ]:
powered2 = pd.DataFrame(powered.pivot_table(index='poweredBy',
                                           values='url',
                                           columns='isHiddenFraudulent'))
powered2.replaceNAs

In [ ]:
df_server = df_rd[df_rd['serverType'].isna()]

In [ ]:
df_server['serverType_New'] = ""
df_server['serverType_New'] = df_server.url.apply(lambda row: getServer(row))
#df_server['serverType_New'] = ""
#df_server.reset_index(inplace=True)

In [ ]:
def getServer(url):
    try:
        temp = requests.get(url)
        temp = temp.headers['server']
    except:
        temp = "NA"
    return temp

In [ ]:
df_server.groupby(['serverType','isHiddenFraudulent']).url.count().reset_index()

In [ ]:
df_server['serverType'] = df_server['serverType_New']
df_server.drop('serverType_New',axis=1,inplace=True)

In [ ]:
x = df_rd[df_rd['serverType'].isna() == False].drop('serverType',axis=1)
x = pd.concat([x,df_server],axis=0)
x

In [ ]:
df_rd2 = df_rd[df_rd['serverType'].isna() == False]
df_rd2 = df_rd2 + pd.concat(df_rd[df_rd['serverType'].isna() == False].drop('serverType',axis=1),df_server['serverType'],axis=1)

In [ ]:
pip install python-whois

# HAS IP COLUMN CODE

In [94]:
import re

In [151]:
pat = re.compile("((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)")
    #"((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)")
    #"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")
def ipCheck(x):
    x = re.sub(r'[^0-9.]', '', x)
    test = pat.match(x)
    if test:
        return 1
    else:
        return 0

In [152]:
df_rd2 = df_rd

In [153]:
df_rd2['hasIP'] = df_rd2.url.apply(lambda row: ipCheck(row))

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [155]:
df_rd2[df_rd2['hasIP']==1].count()

url                   49
compromissionType     49
isHiddenFraudulent    49
contentLength         49
serverType            49
poweredBy             45
contentType           49
lastModified          37
url2                  49
hasIP                 49
dtype: int64

# PRIMARY DOMAIN

In [184]:
import urllib.parse as urlparse
from os.path import splitext, basename



In [272]:
def primaryDomain(x,l='len'):
    temp = urlparse.urlparse(x).hostname
    if l == 'len':
        return len(temp)
    else:
        return temp

In [273]:
t = primaryDomain("http://aaa.bjgdhm.com/show_products.asp",l='str')
t

'bjgdhm.com'

# PARSE URL CHUNKS

In [326]:
cd "C:\Users\arneg\Desktop\Google ML Immersion\URL PROJECT"

C:\Users\arneg\Desktop\Google ML Immersion\URL PROJECT


In [360]:
validExtensions = pd.read_excel("C:/Users/arneg/Desktop/Google ML Immersion/URL PROJECT/Extensions.xlsx")

In [361]:
validExtensions = list(validExtensions['Ext'])

In [412]:
def extFind(url,combine=True):
    t = primaryDomain(url,l='str')
    orig = t
    t = t.split(".")
    temp = ""
    last = t[-1]
    while (len(last) <= 3) & (last in validExtensions):
        if temp == "":
            temp = t.pop()
        else:
            temp = t.pop() + "." + temp
        try:
            last = t[-1]
        except IndexError:
            break
    if combine == True:
        if orig[:2] =="ww":
            t = [t[0],".".join(t[1:])]
        else:
            t = [".".join(t)]
    t.append(temp)
    #print(str(orig) + " --> " + str(t))
    return t

In [414]:
df_rd2 = df_rd

In [416]:
df_rd2['domain_temp'] = ""
df_rd2["domain_temp"] = df_rd2.url.apply(lambda row: extFind(row,True))

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [424]:
df_rd2['domain_temp'].to_csv("C:/Users/arneg/Desktop/Google ML Immersion/URL PROJECT/test.csv")

In [590]:
extFind("http://www.fagnaniebertagnoli.feng.it/VETRINA/index.html",False)

['www', 'fagnaniebertagnoli', 'feng', 'it']

## 88463 www, ww1, ww2  ==> signficant, so believe 3 cols required with 1 hot encoded

In [433]:
x=[1 if x[0] in ['www','ww1','ww2'] else 0 for x in df_rd2['domain_temp']]

In [434]:
sum(x)

88463

In [602]:
def www(x,search):
    temp = 0
    for itm in x[:3]:
        if itm in [search]:
            temp = 1
            break
    return temp

            
def urlVars(df,domain_list_col):
    df['www'] = df.domain_temp.apply(lambda row: www(row,"www"))
    df['ww1'] = df.domain_temp.apply(lambda row: www(row,"ww1"))
    df['ww2'] = df.domain_temp.apply(lambda row: www(row,"ww2"))
    df['URL_Length'] = df.url.apply(lambda row: len(row))
    df['Query'] = df.url.apply(lambda row: urlparse.urlparse(row).query if urlparse.urlparse(row).query != '' else 'None')
    df['Query_Length'] = df.Query.apply(lambda row: len(row))
    df['Primary'] = df[domain_list_col].apply(lambda row: row[1] if row[0] in ['www','ww1','ww2'] else row[0])
    df['Primary_Length'] = df.Primary.apply(lambda row: len(row))
    df['Extension'] = df[domain_list_col].apply(lambda row: row[-1])
    df['Extension_Length'] = df['Extension'].apply(lambda row: len(row))
    df['Num_Periods'] = df.url.apply(lambda row: row.count("."))
    df['Num_Exclam'] = df.url.apply(lambda row: row.count("!"))
    df['Num_Quest'] = df.url.apply(lambda row: row.count("?"))
    df['Num_Perc'] = df.url.apply(lambda row: row.count("%"))
    df['Num_Hyphen'] = df.url.apply(lambda row: row.count("-"))
    df['Num_Numbers'] = df.url.apply(lambda row: sum(c.isdigit() for c in row))
    df['Words'] = df.url.apply(lambda row: "".join((char if char.isalnum() else " ") for char in row).split())
    df['Word_Counts'] = df.Words.apply(lambda row: [len(x) for x in row])
    df['Longest_Word'] = df.Word_Counts.apply(lambda row: max(row))
    df['Shortest_Word'] = df.Word_Counts.apply(lambda row: min(row))
    df['Mean_Word'] = df.Word_Counts.apply(lambda row: float(sum(row))/max(len(row),1))

In [601]:
'1'.isalnum()

True

In [603]:
urlVars(df_rd2,'domain_temp')

C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\arneg\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

[['http',
  'loukilrenard',
  'com',
  'actualites',
  '4',
  'droit',
  'de',
  'la',
  'propriete',
  '3',
  'usucapion',
  'et',
  'modification',
  'de',
  'lassiette',
  'dune',
  'servitude']]

In [536]:
for word in df_rd2['Words']:
    for x in word:
        if (x == "") | (x == None):
            print(word)